In [1]:
!pip install finance-datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 2.9 MB/s eta 0:00:00


In [2]:
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
import numpy as np

df_krx = fdr.StockListing('KRX')
df_krx.head()

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,55100,2,-1200,-2.13,56100,56500,54800,9390914,521335628200,328935018505000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,171900,1,1300,0.76,175000,175100,171400,1341969,232769166100,125143606543500,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,398000,1,4500,1.14,393500,399000,390000,80377,31724876500,93132000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,928000,1,9000,0.98,942000,953000,915000,33323,31000333000,66049472000000,71174000,STK
4,005380,KR7005380001,현대차,KOSPI,,217000,1,1500,0.70,213000,220500,212500,232476,50525487500,45443313447000,209416191,STK


In [4]:
stock_list = ['기아', 'NAVER', '하나금융지주', '삼성전자']
select_df = df_krx[df_krx['Name'].isin(stock_list)].set_index('Code')
select_df.head()

,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
Code,,,,,,,,,,,,,,,,
005930,KR7005930003,삼성전자,KOSPI,,55100,2,-1200,-2.13,56100,56500,54800,9390914,521335628200,328935018505000,5969782550,STK
000270,KR7000270009,기아,KOSPI,,97900,1,2600,2.73,95300,99200,95000,636397,62251365200,39146139024300,399858417,STK
035420,KR7035420009,NAVER,KOSPI,,193700,1,700,0.36,193500,193900,191300,230190,44335420800,31143959199600,160784508,STK
086790,KR7086790003,하나금융지주,KOSPI,,60700,1,800,1.34,59900,61200,59900,283299,17199326100,17435521416000,287240880,STK


In [16]:
import pandas as pd

data = pd.DataFrame()
for code_name, values in select_df.iterrows():
  df = fdr.DataReader(code_name, '2014')[['Close']].rename(columns = {'Close' : values['Name']})
  data = pd.concat([data, df], axis = 1)

In [17]:
Change_df = data.pct_change().dropna()
Change_df.head()

,삼성전자,기아,NAVER,하나금융지주
Date,,,,
2014-01-03,-0.009931,0.000000,-0.035860,-0.031927
2014-01-06,0.008488,-0.003795,0.000000,0.003534
2014-01-07,-0.002295,0.028571,0.001429,-0.025822
2014-01-08,-0.009969,-0.001852,0.000000,0.002410
2014-01-09,-0.012393,-0.027829,-0.041427,0.010817


In [18]:
cum_change_df = (Change_df + 1).cumprod() - 1
cum_change = cum_change_df.iloc[-1]
cum_change

,2024-11-20
삼성전자,1.108480
기아,0.861480
NAVER,0.329155
하나금융지주,0.391106


In [ ]:
# CAGR = (목표금액 / 투자금액) ** (1/투자년도) - 1
# CAGR = (1+누적수익률) ** (1/투자년도) - 1

In [22]:
investment_period = len(cum_change_df) / 252
CAGR = (1+cum_change) ** (1/investment_period) - 1
CAGR

,2024-11-20
삼성전자,0.072802
기아,0.060284
NAVER,0.027168
하나금융지주,0.031586


In [24]:
daily_std = Change_df.std()
annual_std = daily_std * np.sqrt(252)
annual_std

,0
삼성전자,0.259534
기아,0.318805
NAVER,0.341554
하나금융지주,0.324342


   $$ \text{포트폴리오 분산} = \mathbf{w}^T \Sigma \mathbf{w}\ $$

   $$ w = 포트폴리오 내 비중 $$
   $$ \Sigma = 공분산 행렬 $$


  

In [57]:
port_weights_list = []
port_return_list = []
port_std_list = []

for i in range(20000):
  weights = np.random.random(len(stock_list))
  port_weights = weights / np.sum(weights)
  port_return = (port_weights * CAGR).sum()
  covariance_matrix = np.cov(Change_df.T, ddof=0) * 252  # 연간 공분산
  port_std = np.sqrt(port_weights.T @ covariance_matrix @ port_weights)
  port_weights_list.append(port_weights)
  port_return_list.append(port_return)
  port_std_list.append(port_std)

In [58]:
cor_name = Change_df.columns.to_list()

In [59]:
port = {}
port['return'] = port_return_list
port['risk'] = port_std_list

In [60]:
for i in range(len(cor_name)):
  port[cor_name[i]] = [weight[i] for weight in port_weights_list]

In [61]:
port_df = pd.DataFrame(port)
port_df.head()

,return,risk,삼성전자,기아,NAVER,하나금융지주
0,0.048111,0.209114,0.220130,0.284128,0.158904,0.336838
1,0.046222,0.227452,0.041356,0.486087,0.230528,0.242028
2,0.047996,0.204317,0.332104,0.144317,0.321455,0.202123
3,0.054519,0.204415,0.444248,0.176130,0.097820,0.281803
4,0.043749,0.213702,0.105850,0.315516,0.283956,0.294679


In [62]:
import plotly.express as px
fig = px.scatter(port_df, x="risk", y="return",
                 hover_data=port_df.columns[2:])
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [81]:
port_df['sharp'] = port_df['return'] / port_df['risk']
port_df.sort_values('sharp', ascending=False).head(3)

,return,risk,삼성전자,기아,NAVER,하나금융지주,sharp
16909,0.068591,0.224331,0.678709,0.314979,0.001898,0.004413,0.305757
15680,0.067159,0.220254,0.659760,0.298196,0.040233,0.001810,0.304915
13776,0.066872,0.219409,0.659092,0.287067,0.026686,0.027155,0.304782


In [80]:
port_df.sort_values('return', ascending=False).head(3)

,return,risk,삼성전자,기아,NAVER,하나금융지주,sharp
16343,0.069453,0.229947,0.791297,0.185498,0.016121,0.007084,0.302037
4958,0.069296,0.236323,0.853249,0.089949,0.008778,0.048025,0.293228
329,0.068614,0.227547,0.776030,0.180227,0.029224,0.014520,0.301539


In [82]:
port_df.sort_values('risk').head(3)

,return,risk,삼성전자,기아,NAVER,하나금융지주,sharp
18943,0.053161,0.199730,0.400552,0.206135,0.192353,0.200961,0.266165
11905,0.052127,0.199735,0.379237,0.204602,0.217541,0.198619,0.260983
11118,0.052758,0.199735,0.386126,0.212779,0.192104,0.208991,0.264141


In [96]:
data

,삼성전자,기아,NAVER,하나금융지주
Date,,,,
2014-01-02,26180,52700,145205,43850
2014-01-03,25920,52700,139998,42450
2014-01-06,26140,52500,139998,42600
2014-01-07,26080,54000,140198,41500
2014-01-08,25820,53900,140198,41600
...,...,...,...,...
2024-11-14,49900,92500,188500,59800
2024-11-15,53500,91600,190000,59100
2024-11-18,56700,96700,190800,60200


## 라이브러리 활용

In [97]:
!pip install PyPortfolioOpt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.1 MB/s eta 0:00:00


In [99]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

mu = expected_returns.mean_historical_return(data) #returns.mean() * 252
S = risk_models.sample_cov(data) #Get the sample covariance matrix
ef = EfficientFrontier(mu, S)
#Maximize the Sharpe ratio, and get the raw weights
weights = ef.min_volatility() # ef.min_volatility(), ef.max_sharpe() , ef.efficient_return(target_return = 0.2), ef.efficient_risk(0.1)
cleaned_weights = ef.clean_weights()
portfoio_performance = ef.portfolio_performance(verbose=True)

print(cleaned_weights) #Note the weights may have some rounding error, meaning they may not add up exactly to 1 but should be close
print(portfoio_performance)

Expected annual return: 5.3%
Annual volatility: 20.0%
Sharpe Ratio: 0.16
OrderedDict([('삼성전자', 0.38662), ('기아', 0.21494), ('NAVER', 0.20356), ('하나금융지주', 0.19487)])
(0.052790249416731674, 0.19969711289368844, 0.16419991727265493)
